In [12]:
import numpy as np
from ctapipe.io.eventsourcefactory import EventSourceFactory
from ctapipe.calib import CameraCalibrator
from ctapipe.image.hillas import hillas_parameters, HillasParameterizationError
from ctapipe.image.cleaning import tailcuts_clean
from tqdm import tqdm
import pandas as pd
import glob

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
names_to_id = {'LSTCam': 1, 'NectarCam': 2, 'FlashCam': 3, 'DigiCam': 4, 'CHEC': 5}
types_to_id = {'LST': 1, 'MST': 2, 'SST': 3}
allowed_cameras = ['LSTCam', 'NectarCam', 'DigiCam']
allowed_tel_ids = [  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
       174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186,
       187, 188, 189, 190, 191, 192, 193, 194]

In [3]:
def hillas_features(event):
    features = []
    for telescope_id, dl1 in event.dl1.tel.items():
        camera = event.inst.subarray.tels[telescope_id].camera

        mask = tailcuts_clean(camera, dl1.image[0], boundary_thresh=5, picture_thresh=10)
        telescope_type_name = event.inst.subarray.tels[telescope_id].optics.tel_type
        
        image = dl1.image[0].copy()
        image[~mask] = 0
        
        hillas_params = hillas_parameters(
            camera,
            image,
            container=True
        )

        d = hillas_params.as_dict()
        d['telescope_type_name'] = telescope_type_name
        features.append({k: strip_unit(v) for k, v in d.items()})

    return features

def strip_unit(v):
    try:
        return v.si.value
    except AttributeError:
        return v

In [7]:
def analyze_file(input_file, allowed_tel_ids, n_events=-1):
    event_source = EventSourceFactory.produce(
        input_url=input_file,
        max_events=n_events if n_events > 1 else None,
        allowed_tels=allowed_tel_ids
    )

    calibrator = CameraCalibrator(
        eventsource=event_source,
        r1_product='HESSIOR1Calibrator',
    )

    features = []
    for e in tqdm(event_source):
        calibrator.calibrate(e)
        try:
            h = hillas_features(e)
            features.extend(hillas_features(e))
        except HillasParameterizationError:
            continue
    
    return pd.DataFrame(features)

In [14]:
frames=[]
for f in glob.glob('./raw/gamma_*'):
    frames.append(analyze_file(f, allowed_tel_ids))

df = pd.concat(frames)
df.to_csv('gammas.csv', index=False)

['./raw/gamma_20deg_0deg_run485___cta-prod3-merged_desert-2150m-Paranal-3HB89-NGFD.simtel.gz']

In [17]:
frames=[]
for f in glob.glob('./raw/proton_*'):
    print(f)
    frames.append(analyze_file(f, allowed_tel_ids))

df = pd.concat(frames)
df.to_csv('protons.csv', index=False)


0it [00:00, ?it/s]

./raw/proton_20deg_0deg_run10___cta-prod3-merged_desert-2150m-Paranal-3HB89-NGFD.simtel.gz



1it [00:00,  1.03it/s]
2it [00:01,  1.62it/s]
3it [00:01,  2.11it/s]
4it [00:01,  2.41it/s]
5it [00:01,  2.79it/s]
7it [00:02,  3.15it/s]
8it [00:02,  3.30it/s]/Users/Kai/Development/ctapipe/ctapipe/image/hillas.py:571: RuntimeWarning: invalid value encountered in sqrt
  width = np.sqrt((vy2 + vx2 - z) / 2.0)

9it [00:02,  3.45it/s]
10it [00:02,  3.37it/s]
11it [00:03,  3.56it/s]
13it [00:03,  4.07it/s]
15it [00:03,  4.41it/s]
16it [00:03,  4.54it/s]
17it [00:03,  4.66it/s]
18it [00:03,  4.54it/s]
19it [00:04,  4.50it/s]
21it [00:04,  4.73it/s]
22it [00:04,  4.64it/s]
23it [00:04,  4.73it/s]
25it [00:05,  4.87it/s]
26it [00:05,  4.94it/s]
28it [00:05,  5.10it/s]
29it [00:05,  5.06it/s]
30it [00:06,  4.92it/s]
31it [00:06,  5.00it/s]
32it [00:06,  5.06it/s]
34it [00:06,  5.25it/s]
35it [00:06,  5.27it/s]
37it [00:06,  5.35it/s]
38it [00:07,  5.35it/s]
40it [00:07,  5.53it/s]
41it [00:07,  5.57it/s]
42it [00:07,  5.60it/s]
44it [00:07,  5.72it/s]
46it [00:07,  5.79it/s]
47it [00:08,  5.

./raw/proton_20deg_0deg_run11___cta-prod3-merged_desert-2150m-Paranal-3HB89-NGFD.simtel.gz



1it [00:00,  1.42it/s]
2it [00:00,  2.18it/s]
5it [00:01,  4.60it/s]
6it [00:01,  4.71it/s]
7it [00:01,  4.73it/s]
9it [00:01,  5.00it/s]
11it [00:01,  5.72it/s]
12it [00:02,  5.51it/s]
14it [00:02,  5.84it/s]
15it [00:02,  5.96it/s]
16it [00:02,  6.06it/s]
17it [00:02,  5.90it/s]
18it [00:03,  5.93it/s]
20it [00:03,  5.96it/s]
22it [00:03,  6.03it/s]
24it [00:03,  6.31it/s]
25it [00:03,  6.31it/s]
27it [00:04,  6.56it/s]
29it [00:04,  6.81it/s]
31it [00:04,  7.03it/s]
33it [00:04,  7.21it/s]
36it [00:04,  7.50it/s]
38it [00:05,  7.56it/s]
40it [00:05,  7.76it/s]
42it [00:05,  7.97it/s]
44it [00:05,  8.09it/s]
46it [00:05,  8.11it/s]
48it [00:06,  7.90it/s]
49it [00:06,  7.63it/s]
51it [00:07,  7.28it/s]
52it [00:07,  7.25it/s]
53it [00:07,  7.21it/s]
55it [00:07,  7.31it/s]
56it [00:07,  7.30it/s]
58it [00:07,  7.29it/s]
59it [00:08,  7.21it/s]
60it [00:08,  7.18it/s]
61it [00:08,  7.06it/s]
62it [00:09,  6.89it/s]
63it [00:09,  6.85it/s]
64it [00:09,  6.84it/s]
65it [00:09,  6.87it/